In [183]:
import altair as alt
import pandas as pd

In [184]:
# %load const
CO_STO_AGOS_NL = 33087679
CO_STO_AGO_INST = 33176825
CO_STO_AGO = 33020566
CO_SAO_BENTO = 33062633
CO_SAO_VICENTE = 33063648
CO_PARQUE = 33065837
CO_ELEVA = 33178860
CO_ST_INACIO = 33063729

CO_MUN = 3304557  # Rio de Janeiro
CO_UF_RIO = 33  # RJ

MIN_ALUNOS = 30  # Mínimo de alunos fazendo Enem para considerar na análise
NUM_MELHORES = 60
ANO_ULT = 2019  # último ano a processar

## Colégio a processar
Mudar abaixo para ver outros colégios

In [185]:
CO_PROCESSAR = CO_PARQUE

In [186]:
df_turmas_rio = pd.read_feather("dados/turmas_rio.feather")

In [187]:
df_turmas_rio.ano.max()

2019

In [188]:
em_etapas = range(25, 39)  # são vicente deveria ter só 25-27
df_medio = df_turmas_rio.query(
    "id_etapa.isin(@em_etapas) and id_escola==@CO_PROCESSAR"
)

In [189]:
df_medio_por_ano = df_medio.groupby("ano").agg(
    total=("num_matriculas", "sum"), no_turmas=("index", "count")
)
(
    (
        alt.Chart(df_medio_por_ano.reset_index())
        .mark_line()
        .encode(
            y=alt.Y(
                "total",
                title=None,
                scale=alt.Scale(
                    domain=[
                        df_medio_por_ano.total.min()
                        - (df_medio_por_ano.total.max() - df_medio_por_ano.total.min())
                        / 2,
                        df_medio_por_ano.total.max(),
                    ]
                ),
            ),
            x=alt.X(
                "ano:O",
                title=None,
                axis=alt.Axis(labelAngle=0, domain=False),
            ),
        )
        .properties(
            width=500,
        )
    )
    + alt.Chart(df_medio_por_ano.mean().to_frame().T)
    .mark_line(color="red")
    .encode(y="total")
    + alt.Chart(df_medio_por_ano.mean().to_frame().T)
    .mark_rule(color="orange")
    .encode(y="total")
)

alt.LayerChart(...)

## Perguntas
1. como tem variado o número de alunos do primeiro para o terceiro ano

In [194]:
df_medio_sv = df_medio[["ano", "id_etapa", "num_matriculas"]].groupby(["ano", "id_etapa"]).aggregate(
    num_matriculas=("num_matriculas", sum), num_turmas=("num_matriculas", "count")
).rename({25:'1º', 26: '2º', 27: '3º'}).reset_index().rename(columns={'id_etapa': 'Série'})#.unstack()['num_matriculas'].rename(columns={25:'1º ano', 26: '2º ano', 27: '3º ano'}).reset_index()
df_medio_sv.head()

,ano,Série,num_matriculas,num_turmas
0,2007,1º,114,3
1,2007,2º,112,4
2,2007,3º,84,3
3,2008,1º,124,4
4,2008,2º,101,3


In [191]:
alt.Chart(df_medio_sv).mark_line().encode(x='ano', y='num_matriculas', color='Série', tooltip=('num_matriculas', 'num_turmas'))

alt.Chart(...)

In [192]:
df_sv_periodo = df_medio_sv.pivot(
    index="ano", columns="Série", values="num_matriculas"
).reset_index()
df_sv_periodo["2º"] = df_sv_periodo["2º"].shift(-1)
df_sv_periodo["2º %"] = (df_sv_periodo["2º"]/df_sv_periodo["1º"] * 100).round(0)
df_sv_periodo["3º"] = df_sv_periodo["3º"].shift(-2)
df_sv_periodo["3º %"] = (df_sv_periodo["3º"]/df_sv_periodo["1º"] * 100).round(0)
df_sv_periodo["1º %"] = 100
df_sv_periodo["Período"] = (
    df_sv_periodo["ano"].astype(str) + "–" + (df_sv_periodo["ano"] + 2).astype(str)
)
df_sv_periodo_no = (
    df_sv_periodo.drop("ano", axis=1)
    .melt(id_vars=["Período"], value_vars=["1º", "2º", "3º"], var_name="Série", value_name='Nº alunos')
    .sort_values("Período")
)
df_sv_periodo_per = (
    df_sv_periodo.drop("ano", axis=1)
    .melt(id_vars=["Período"], value_vars=["1º %", "2º %", "3º %"], var_name="Série", value_name='Nº alunos')
    .sort_values("Período")
)

df_sv_periodo_per.head()

,Período,Série,Nº alunos
0,2007–2009,1º %,100.0
13,2007–2009,2º %,89.0
26,2007–2009,3º %,69.0
1,2008–2010,1º %,100.0
14,2008–2010,2º %,89.0


In [193]:
def line_chart(df):
    return (
        alt.Chart(df)
        .mark_line(interpolate="monotone", point=True)
        .encode(
            x="Série",
            y=alt.Y("Nº alunos"),#, scale=alt.Scale(zero=False)),
            color="Período:O",
            tooltip=('Período', "Nº alunos")
        )
        .properties(width=800)
    )


line_chart(df_sv_periodo_no) & line_chart(df_sv_periodo_per)

alt.VConcatChart(...)

2. como tem variado as notas dos alunos no enem (a princípio seriam comparáveis)

In [236]:
df_enem = pd.read_feather("dados/enem_anos.feather")

In [239]:
df_enem.head()

,CO_ESCOLA,Ano,nota_ciências,nota_humanas,nota_português,nota_matemática,nota_redação,nota_final,num_alunos,nota_ciências_perc80,nota_humanas_perc80,nota_português_perc80,nota_matemática_perc80,nota_redação_perc80,nota_final_perc80,rank
0,33135827,2019,647.5,651.0,627.0,789.0,940.0,725.0,31.0,676.0,685.5,641.5,823.0,960.0,746.0,1
1,33062633,2019,636.5,672.0,626.0,789.5,900.0,722.0,45.0,662.5,714.0,641.5,853.0,940.0,743.0,2
2,33178860,2019,610.0,662.5,626.0,779.0,910.0,711.5,60.0,652.5,683.5,648.0,848.0,940.0,744.0,3
3,33176825,2019,605.5,649.0,620.5,759.0,920.0,707.0,146.0,643.0,682.0,644.5,815.5,960.0,735.0,4
4,33062609,2019,611.5,649.5,612.5,777.5,880.0,707.0,80.0,649.5,682.0,632.0,831.0,920.0,738.0,4


In [243]:
alt.Chart(
    df_enem[df_enem.CO_ESCOLA.isin([CO_SAO_VICENTE])]
    # df_enem.query("rank <= 30 and NU_ANO == @ANO_ULT").CO_ESCOLA)]
).mark_line(point=True).encode(
    x="Ano:O",
    y=alt.Y("nota_matemática"),
    color="CO_ESCOLA:N",
    tooltip=("rank", "nota_final", "nota_final_perc80", 'nota_matemática'),
).properties(
    width=800
)

alt.Chart(...)

In [253]:
materias = alt.Chart(
    df_enem[df_enem.CO_ESCOLA == CO_SAO_VICENTE].melt(
        id_vars="Ano",
        value_vars=[c for c in df_enem.columns if c.startswith("nota_") and not c.endswith('80')],
        var_name="Matéria",
        value_name="Nota",
    )
).mark_line(point=True).encode(
    x="Ano:O",
    y=alt.Y("Nota", scale=alt.Scale(zero=False)),
    color="Matéria", tooltip=[ "Matéria", "Nota","Ano"]
    #tooltip=(f'{i}:Q' for i in df_enem.columns if i.startswith('nota_')),
).properties(
    width=800
)
materias

alt.Chart(...)

In [230]:
df_enem[df_enem.CO_ESCOLA == CO_SAO_VICENTE].melt(
        id_vars="NU_ANO",
        value_vars=[c for c in df_enem.columns if c.startswith("NU")],
        var_name="Matéria",
        value_name="Nota",
    )

,NU_ANO,Matéria,Nota
0,2019,NU_NOTA_CN,588.0
1,2018,NU_NOTA_CN,598.0
2,2017,NU_NOTA_CN,620.0
3,2016,NU_NOTA_CN,628.5
4,2015,NU_NOTA_CN,629.5
5,2014,NU_NOTA_CN,625.5
6,2013,NU_NOTA_CN,618.0
7,2019,NU_NOTA_CH,635.5
8,2018,NU_NOTA_CH,673.5
9,2017,NU_NOTA_CH,654.0


3. em que percentil tem ficado as notas?